In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import nltk
import matplotlib.pyplot as plt
import os
import pymongo
import pandas as pd
import re
import seaborn as sns
import time

In [2]:
# use pymongo to store all info from LinkedIN
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.linkedin_db

In [12]:
# chrome_driver_path = os.path.abspath(r"C:\Users\haneu\Desktop\Data Analytics\6_mongo_webScrape\chromedriver.exe")
executable_path = {'executable_path': "chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.glassdoor.ca/index.htm"
browser.visit(url)

In [13]:
print("What job are you looking for?")
job = input()
job_type = browser.find_by_id("KeywordSearch")
job_type.fill(job)

print("Where do you want to find your job?")
job_location = input()
location = browser.find_by_id("LocationSearch")
location.fill(job_location)

# Clicking button
browser.find_by_id("HeroSearchButton").click()

What job are you looking for?
data analyst
Where do you want to find your job?
korea


In [14]:
# list to store scraped data
company = []
all_location = []
job_desc = []
position = []

def scrape_current_page():

    # Getting html of first page
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    jobs = soup.find_all("li", class_="jl")

    for job in jobs:
        # Store all info into a list         
        position.append(job.find("div", class_="jobTitle").a.text)
        
        # Pop up will show up as soon as we start scarping.
        # so wait few seconds and exit pop-up
        print("count down")
        time.sleep(5)
        

        exit_button = soup.find("div", class_="ModalStyle__xBtn___34qya")
        print(f'exit_button{exit_button}')
        # If there was a pop-up click close button.
        if exit_button:
            browser.find_by_css(".ModalStyle__xBtn___34qya").first.click()
        
        
        # ex: Tommy - Singapore
        comp_loc = job.find("div", class_="empLoc").div.text
        comp, loc = comp_loc.split("–")
        # print(comp)
        company.append(comp.strip())
        all_location.append(loc.strip())
                
        # ------------- Scrape Job descriptions within a page -----------
        # job description is in another html, therefore retrieve it once again after
        # clicking.
        browser.click_link_by_href(job.find("a", class_="jobLink")["href"])
        html = browser.html
        soup = BeautifulSoup(html, "html.parser")
        job_desc.append(soup.find("div", class_="desc").text)

        time.sleep(1) # Since splinter scrapes too fast and skips some job description.
    return None

def scrape_all():
    # grab new html, grab page control elements
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    # Will throw an error if there is no pagining control => one page => goto except statement
    try:
        result = soup.find("div", class_="pagingControls").ul
        pages = result.find_all("li")

        for page in pages:
            # scrape each page
            scrape_current_page()
            # run if <a> exists since un-clickable do not have <a> skipping < and pg1
            if page.a:
                # within <a> tag click except next button         
                if not page.find("li", class_="Next"):
                    try:
                        # Click to goto next page.
                        browser.click_link_by_href(page.a['href'])
                    except:
                        print("This is the last page")
                        break
    # only one page
    except:
        scrape_current_page()

    # No need to return since we appened all data into list
    return None

In [15]:
scrape_all()

count down
exit_buttonNone
count down
exit_button<div class="ModalStyle__xBtn___34qya">✕</div>
count down
exit_buttonNone
count down
exit_buttonNone
count down
exit_buttonNone
count down
exit_buttonNone
count down
exit_buttonNone
count down
exit_buttonNone
count down
exit_buttonNone
count down
exit_buttonNone


In [18]:
job_desc

[' Data Analytics & Visualization Analyst\n\nADW, oracle Analytic cloud\n\n Summary\n\nDesigns analytics for strategic use across the line of business. Design/Create reports and analytics that provide key business insights used for data driven decision making.\n\nPossesses a strong business insight coupled with technical ability to know how to develop, explore, analyze, and present data and solutions to a line of business audience.\n\n Responsibility\n\n Need a proactive, consultative approach to analytics. Asks the right questions of the pain point and makes proactive recommendations aligned to overall business strategy\n Analyze data from multiple data sources and design and create reports\n Communicates and presents analysis and insights to business customers and executive management.\n Evaluates business needs and improves efficiencies and capabilities through the use of technology.\n Need train personnel in scope and functionality of processes, systems, and solutions.\n\n Job Code

In [16]:
# checking duplicate scrape
# Total 30 jobs in each page.
print(len(job_desc))
print(len(set(job_desc)))
job_desc
# get list of it and turn it back to a list. 
# job_desc = set(job_desc)
# job_desc = list(job_desc)

10
7


[' Data Analytics & Visualization Analyst\n\nADW, oracle Analytic cloud\n\n Summary\n\nDesigns analytics for strategic use across the line of business. Design/Create reports and analytics that provide key business insights used for data driven decision making.\n\nPossesses a strong business insight coupled with technical ability to know how to develop, explore, analyze, and present data and solutions to a line of business audience.\n\n Responsibility\n\n Need a proactive, consultative approach to analytics. Asks the right questions of the pain point and makes proactive recommendations aligned to overall business strategy\n Analyze data from multiple data sources and design and create reports\n Communicates and presents analysis and insights to business customers and executive management.\n Evaluates business needs and improves efficiencies and capabilities through the use of technology.\n Need train personnel in scope and functionality of processes, systems, and solutions.\n\n Job Code

In [ ]:
# ------------- Text classification to classify technical skill words --------------
# Some words are connected with / ex:"sql/database", so split them.
for job in job_desc:
    ", ".join(job.split('/'))
job_desc = [", ".join(job.split('/')) for job in job_desc]

# each item is a list of tokenized job_descriptions
tok = [nltk.word_tokenize(job.lower()) for job in job_desc]

from nltk.corpus import stopwords
stop = stopwords.words('english')
def stopword_deleter(tokenized_job_desc):
    """ ignore stop words, bullets, etc. And put it into one list """
    final_word_list = []
    for lists in tokenized_job_desc:
        for item in lists:
    # --------------------------------- Use REGEX to exclude numbers ------------------
            if len(item)>2 and (item not in stop) and (item.isalpha()):
                # Some words have \\ at the end, remove them.           
                final_word_list.append(item.replace("\\",""))
    return final_word_list


# ------------ Lematize
cleaned_list = stopword_deleter(tok)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_list = [lemmatizer.lemmatize(word,pos="v") for word in cleaned_list]

In [ ]:
print(tok)

In [ ]:
cleaned_list

In [ ]:
# -------------Taking a look at POS tags -(NOT USED)
pos_tag = nltk.pos_tag(lemmatized_list)
pos_df = pd.DataFrame(pos_tag, columns=["Word", "POS"])
pos_sum = pos_df.groupby("POS").count()
pos_sum.sort_values(["Word"], ascending=False)

filtered_pos_tag = []
for tag in pos_tag:
    if tag[1] =="NN" or tag[1] == "NNS" or tag[1] =="NNP" or tag[1] == "NNPS":
        filtered_pos_tag.append(tag)

In [ ]:
# Find 100 most frequent words from all scraped job_description
freq = nltk.FreqDist(lemmatized_list)
most_freq_words = freq.most_common(100)
most_freq_words

In [ ]:
df = pd.DataFrame(most_freq_words, columns=("Words", "Count"))
df.head()

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stop,
                          max_words=100,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(most_freq_words))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(cleaned_corpus, n=100):
    vec = CountVectorizer().fit(cleaned_corpus)
    bag_of_words = vec.transform(cleaned_corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]

top_words = get_top_n_words(lemmatized_list, n=100)
# top_df = pd.DataFrame(top_words)

In [ ]:
top_words[:3]

In [ ]:
my_list = [1,2,3]

In [ ]:
new_dict = {}
new_list = []
for key,value in top_words:
    new_dict[key] = value
new_list.append(new_dict)
print(new_list)

In [ ]:
final = []
final.append(new_list)
final.append(my_list)
print(final)

In [ ]:
top_df.head()

In [ ]:
# For 2-grams
def get_top_n2_words(corpus, n=None):
    # Consider 2-grams grabbing top 2000 most occuring term
    vec1 = CountVectorizer(ngram_range=(1,2),
            max_features=2000).fit(corpus) 
    bag_of_words = vec1.transform(corpus) # Create sparce marix.
    sum_words = bag_of_words.sum(axis=0)
    print(sum_words[0])
#     print(vec1.vocabulary_.items())
    words_freq = [(word, sum_words[0, idx]) for word, idx in #Select 0 because dict_items has all tuples in a first list., ("job", 971) tuple is one item, then as idx increase it select next tuple and so on.
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True) #Sort by number. since ("job", 93). x[1] = 93. In descending order.
    return words_freq[:n]
top2_words = get_top_n2_words(lemmatized_list, n=100)
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]


In [ ]:
top2_df.head(15)

In [ ]:
# Visualize
top_words = get_top_n2_words(lemmatized_list, n=30)
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
sns.set(rc={'figure.figsize':(13,8)})
g = sns.barplot(x="Word", y="Freq", data=top_df)
g.set_title("1,2-gram words")
g.set_xticklabels(g.get_xticklabels(), rotation=45);